# Objetivo

O objetivo desse tutorial é introduzí-los a coloeta de dados na web utilizando a ferramenta Scrapy.

Vamos mostrar um exemplo simples de coleta utilizando o scrapy já pronto, e etão vocês vão ter de construir suas próprias "aranhas" (spiders) e parses para realizarem a coleta em um site com dados do IBGE que criamos especialmente para esssa aula.


# Scrapy

É um **framework** de código aberto e colaborativo para extração de dados a partir de websites. Ele escala bem, é simples de utilizar e facilmente extensível.

## Visão Geral da Arquitetura

A figura a seguir mostra uma visão geral da arquitetura do Scrapy com seus componentes e o fluxo dos dados nele [[1]](#[1]-Documentação-sobre-Arquitetura-do-Scrapy).

![Arquitetura do Scrapy](imgs/scrapy_architecture_02.png)

1. A *Engine* recebe a requisição inicial da Spider para inciar o ***crawl***
2. A *Engine* agenda as requisições no ***Scheduler***(Escalonador) e pede a próxima requisição para ***crawl***.
3. O Escalonador retorna a próxima Requisição para a *Engine*.
4. A Engine enviar as *requisições* ao ***Downloader***, passando pelo ***Downloader Middlewares***.
5. Assim que a página foi baixada o *Downloader* gera a Resposta (com aquela página) e envia para a Engine, passando pelo o *Downloader Middlewares*.
6. A *Engine* recebe a Resposta do *Downloader* e envia para a ***Spider*** para processamento, passando pela ***Spider Middleware*** .
7. A *Spider* processa a Resposta e retorna os items extraídos e um nova Requisição é feita à Engine, passando pela *Spider Middleware*.
8. A *Engine* envia os items processados ao ***Item Pipelines***, então envia as Requisições processadas ao Escalonador e pede possíveis novas requisições para *crawl*.
9. O processo é repetido (a partir do passo 1) até que não haja requisições do Escalonador.


## Rodando nossa primeira Spider

No desktop da máquina virtual tem uma pasta ***hands-on***, a mesma contém uma pasta crawler. Vamos acessá-la:

```bash
$ cd ~/Desktop/hands-on
# verifique o conteúdo
~/Desktop/hands-on$ ls

crawler
```
Entremos na pasta crawler, nela tem um exemplo simples de uma Spider.

```bash
~/Desktop/hands-on$ cd crawler
~/Desktop/hands-on/crawler$ ls
example_spider.py
```

```bash
~/Desktop/hands-on$ cd crawler
~/Desktop/hands-on/crawler$ ls
example_spider.py
```

O arquivo **example_spider.py** contém o seguinte código:
```python
import scrapy

class BlogSpider(scrapy.Spider):
    name = 'blogspider'
    start_urls = ['https://blog.scrapinghub.com']

    def parse(self, response):
        for title in response.css('h2.entry-title'):
            yield {'title': title.css('a ::text').extract_first()}

        for next_page in response.css('div.prev-post > a'):
            yield response.follow(next_page, self.parse)
```

Vamos destrinchar o código acima. Primeiramente, o scrapy já possui uma classe base para criar objetos do tipo ```scrapy.Spider```.
Portanto, nós temos de criar uma classe para nossa Spider que herda dessa classe, como está no trecho:

```python
import scrapy

class BlogSpider(scrapy.Spider):
...
```
 Fazendo isso, nós estamos criando uma classe que respeita a interface esperada para execução do fluxo da arquitetura descrito anteriormente. Para isso, é necessário sobre escrever o método ```parse(self, response)``` da classe ```scrapy.Spider```. 

```python
    def parse(self, response):
        for title in response.css('h2.entry-title'):
            yield {'title': title.css('a ::text').extract_first()}

        for next_page in response.css('div.prev-post > a'):
            yield response.follow(next_page, self.parse)
``` 
Esse método é responsável pela leitura da página e extração dos dados. Como podemos ver na chamada do método, esse método recebe como paramentro um response que contém a página web, além disso nos permite percorrer o [DOM do html](https://www.w3schools.com/js/js_htmldom.asp) de forma bem simples (como veremos a seguir).

```python
    for title in response.css('h2.entry-title'):
```

O objeto ```response``` possui os Selectors (ou Seletores) [[2]](#[2]-Selectors) que nos ajudam na tarefa de percorrer o DOM do html e extrair os dados elementos espefícos, e isso é possível utilizando expressões XPath ou CSS. O trecho de cigo acima irá retornar todas as ocorrências da tag ```<h2 class='entry-title">``` da página atual. Para cada ocorrência o código a seguir é executado : 
```python 
        yield {'title': title.css('a ::text').extract_first()}
```

O como ```yield``` irá enviar um item ao ***Item Pipeline*** do Scrapy (como mostrado no fluxo anteriormente). Esse Item pode ser tanto um ```dict``` do python ou um objeto da classe ```scrapy.Item```. Nesse trecho de código, é utilizado o dicionário do python, onde é extraído uma única informação da página que é o texto do link (```<a> algum texto </a>```). São todos o links da página? Não, são apenas links dentro da tag ```<h2 class='entry-title">``` que é representado pelo objeto ```title```.


O com o comando ```yield``` também é possível enviar para o Escalonador novas páginas para serem analisadas. O treco a seguir faz justamente isso.

```python
           for next_page in response.css('div.prev-post > a'):
                yield response.follow(next_page, self.parse)
``` 

Para cada link ```response.css('div.prev-post > a')``` dentro da tag ```<div class="prev-post">``` é utilizado o método follow que vai gerar uma requisição ao Escalonador para cada link e essa requisição utilizará a própria função ```parse``` definida previamente.

Mas de qual página ele vai iniciar a varredura? No início da definição de nossa classe é configurado quais serão as urls de partida.

```python
import scrapy

class BlogSpider(scrapy.Spider):
    name = 'blogspider'
    start_urls = ['https://blog.scrapinghub.com']
```

Além disso, também é uma boa prática definir um nome para nossa Spider.

Agora que já deu para ter uma ideia de como funciona o Scrapy e como extraimos informações de páginas web com ele, vamos rodar nossa Spider e ver ela em ação.


```bash
cd ~/Desktop/hands-on/crawler
scrapy runspider example_spider.py -o items.json

2017-06-07 11:36:59 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2017-06-07 11:36:59 [scrapy.utils.log] INFO: Overridden settings: {'FEED_URI': 'items.json', 'FEED_FORMAT': 'json', 'SPIDER_LOADER_WARN_ONLY': True}
2017-06-07 11:36:59 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.feedexport.FeedExporter']
2017-06-07 11:36:59 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',
 'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware',
 'scrapy.downloadermiddlewares.stats.DownloaderStats']
2017-06-07 11:36:59 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.spidermiddlewares.referer.RefererMiddleware',
 'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',
 'scrapy.spidermiddlewares.depth.DepthMiddleware']
2017-06-07 11:36:59 [scrapy.middleware] INFO: Enabled item pipelines:
[]
2017-06-07 11:36:59 [scrapy.core.engine] INFO: Spider opened
2017-06-07 11:36:59 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2017-06-07 11:36:59 [scrapy.extensions.telnet] DEBUG: Telnet console listening on 127.0.0.1:6023
2017-06-07 11:37:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://blog.scrapinghub.com> (referer: None)
2017-06-07 11:37:01 [scrapy.core.scraper] DEBUG: Scraped from <200 https://blog.scrapinghub.com>

...

2017-06-07 11:37:04 [scrapy.core.engine] INFO: Closing spider (finished)
2017-06-07 11:37:04 [scrapy.extensions.feedexport] INFO: Stored json feed (103 items) in: items.json
2017-06-07 11:37:04 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 2933,
 'downloader/request_count': 11,
 'downloader/request_method_count/GET': 11,
 'downloader/response_bytes': 123796,
 'downloader/response_count': 11,
 'downloader/response_status_count/200': 11,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2017, 6, 7, 14, 37, 4, 796248),
 'item_scraped_count': 103,
 'log_count/DEBUG': 115,
 'log_count/INFO': 8,
 'memusage/max': 48447488,
 'memusage/startup': 48447488,
 'request_depth_max': 10,
 'response_received_count': 11,
 'scheduler/dequeued': 11,
 'scheduler/dequeued/memory': 11,
 'scheduler/enqueued': 11,
 'scheduler/enqueued/memory': 11,
 'start_time': datetime.datetime(2017, 6, 7, 14, 36, 59, 917935)}
2017-06-07 11:37:04 [scrapy.core.engine] INFO: Spider closed (finished)
```

# Desafio

Esse exemplo foi apenas um introdutório a fim de dar uma visão geral do funcionamento do Scrapy. Agora nós iremos coletar os dados que iremos utilizar no restante desse *hands on*. 

## O site

Nós vamos extrair os dados de um site fictício que contém alguns dados do censo do IBGE de 2010. Para acessar esse site, abra navegador e digite na barra de endereço:

```http://127.0.0.1:5000```

Você deveria ver a seguinte página

![Site com Censo 2010](imgs/site.png)

Navegue rapidamente nas páginas... Nós estamos interessados nos seguintes dados:

![Dados de Interesse](imgs/dados_interesse.png)

Clique com botão direito do mouse em qualquer parte da página e deve abrir um popup igual a imagem:

![Dados de Interesse](imgs/inspect_element.png)

Clique em **Inspect Element (Q)**, ira abrir um inspetor de elementos do navegador. Como exibido a seguir:

![Dados de Interesse](imgs/inspetor.png)

Com inspetor de elementos nós conseguimos ver a hieraquia do HTML, e assim, pensar em Selector CSS para navegação da Spider pelos links e extração dos dados de interesse.

## Projeto para Scrapy

Vamos criar um projeto Scrapy para fazermos a coleta. Rodando o seguinte comando criamos nosso projeto.
```bash
# lembre-se de entrar na pasta crawler primeiro
cd ~/Desktop/hands-on/crawler

# comando para criar um projeto scrapy
scrapy startproject ibge
```
Você deveria obter o seguinte resultado:

```bash
New Scrapy project 'ibge', using template directory '/home/class/Desktop/hands-on/.venv/lib/python3.5/site-packages/scrapy/templates/project', created in:
   /home/class/Desktop/hands-on/crawler/ibgeYou can start your first spider with:
   cd ibge
   scrapy genspider example example.com
```
Portanto, nós acabamos de criar um projeto chamado **ibge**. Esse projeto consiste de uma pasta com arquivos referentes ao processo de crawling executado pelo Scrapy. Vamos averiguar a pasta:

```bash
ibge/
    scrapy.cfg
    ibge/
        __init__.py
        middlewares.py
        pipelines.py
        settings.py
        spiders/
                __init__.py
```

Como já havíamos falado, o scrapy é altamente flexível desse modo ele nos permite confirguramos (settings.py), e estendermos vários aspectos (pipelines.py e middlewares.py). Além disso, podemos notar que há uma pasta ```spiders/``` e a mesma está vazia (exceto pelo arquivo ```__init__.py```). De fato, nós ainda não temos nenhuma Spider. Precisamos criar uma para fazermos a coleta.

Nós temos duas opções para criar uma Spider, 1. podemos fazer na mão e 2. utilizando outro comando o scrapy que gera um Spider padrão. Vamos optar pela segunda:

```bash
cd ibge
scrapy genspider censo 127.0.0.1:5000
```

Como resultado do comando temos:

```bash
Created spider 'censo' using template 'basic' in module:
 ibge.spiders.censo
 
ibge/
    scrapy.cfg
    ibge/
        __init__.py
        middlewares.py
        pipelines.py
        settings.py
        spiders/
                __init__.py
                censo.py
```

Foi gerado um arquivo dentro da pasta ```spiders/``` contendo nossa Spider responsável por coletar os dados do Censo do IBGE do site ```http://127.0.0.1:5000/```. Vamos dar uma olhada no arquivo criado:

```python
# -- coding: utf-8 --

import scrapy

class CensoSpider(scrapy.Spider):
   name = 'censo'
   allowed_domains = ['127.0.0.1:5000']
   start_urls = ['http://127.0.0.1:5000/']
   
   def parse(self, response):
       pass
```

O comando ```scrapy genspider censo 127.0.0.1:5000``` nos gerou uma Spider chamada **censo** e com a url inicial sendo o endereço que passamos **http://127.0.0.1:5000**. Como pode ser visto, nós temos de implementar o método da ```parse``` de acordo com nossa necessidade. Como fora mencionado, é nela que nós definimos como a Spider vai navegar pelos links e extrair as informações de interesse.



## Exercício 1

Uma vez que temos uma casca para nossa Spider, peço para que você implmentem o método ```parse(slef, response)``` com objetivo de extrair nossas dados de interesse para cada estado, município e área. Os dados de interesse são:

![Dados de Interesse](imgs/dados_interesse.png)

Além desses exibidos na figura, também queremos o identificador e nome dos estados, municípios e áreas.

***Dica 1*** *: Utilizar o arquivo ```example_spider.py``` como espelho*

***Dica 2*** *: remover ```allowed_domains =  ['127.0.0.1:5000']``` ou então tire a porta, senão a Spider não vai funcionar muito bem*

***Dica 3*** *: Crie outros métodos se necesário*

# Referências

###### [1] [Documentação sobre Arquitetura do Scrapy](https://docs.scrapy.org/en/latest/topics/architecture.html)
###### [2] [Selectors](https://docs.scrapy.org/en/latest/topics/selectors.html)
###### [3] [Documentação Scrapy](https://docs.scrapy.org/en/latest/)